In [ ]:
import os 
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import time

In [ ]:
base_url = "https://capitalwholesalediamonds.com/product-category/round/"
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')


In [ ]:
if not os.path.exists('web_scraped/round/images'):
    os.makedirs('web_scraped/round/images')

This is to add all the summary data to the csv 


In [ ]:
product_divs = soup.find_all('div', class_ = 'ftc-product product')

for product_div in product_divs:
    
    product_url = (product_div.find('a')).get('href')
    response = requests.get(product_url)
    product_page = BeautifulSoup(response.content, 'html.parser')

summary_div = product_page.find('div', class_ ="panel entry-content wc-tab")
summary_txt = summary_div.find('p').get_text()

# Split the text content into lines
lines = summary_txt.split('\n')

# Create a dictionary to store the extracted data
data = {}
for line in lines:
    if line:
        key, value = line.split(':', 1)
        data[key.strip()] = value.strip()

csv_file_path = "/home/levi/Desktop/Work/Diamond Project/web_scraped/round/data_round.csv"

# Write the data into a CSV file
with open(csv_file_path, mode='a', newline='') as csv_file:
    fieldnames = data.keys()
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    # writer.writeheader()
    writer.writerow(data)

print(f"Data saved to {csv_file_path}")

This is to store the image files with the appropriate name corresponding to their Stock Number


In [ ]:
product_divs = soup.find_all('div', class_ = 'ftc-product product')

for product_div in product_divs:
    
    product_url = (product_div.find('a')).get('href')
    response = requests.get(product_url)
    product_page = BeautifulSoup(response.content, 'html.parser')

img_url = product_page.find('div', class_ = "iconic-woothumbs-images-wrap").find('img').get('src')

# Send a request to the image URL
img_response = requests.get(img_url)

# Get the image data
img_data = img_response.content

# to do it in one line we can employ it as img_data = requests.get(product_page.find('div', class_ = "iconic-woothumbs-images-wrap").find('img').get('src')).content

# # Save the image to a local file
with open((f"web_scraped/round/images/{data['Stock Number']}.jpg") , "wb") as img_file:
    img_file.write(img_data)

print("Image download and saving completed.")

summary_div = product_page.find('div', class_ ="panel entry-content wc-tab")

Summarizing both of them together to iterate through the whole page. We convert this in to function call. Make sure to initialize the csv headers according to the summary data


In [ ]:
csv_file_path = "/home/levi/Desktop/Work/Diamond Project/web_scraped/round/data_round.csv"


def page_iterator(base_url):

    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    product_divs = soup.find_all('div', class_ = 'ftc-product product')

    for product_div in product_divs:
        
        product_url = (product_div.find('a')).get('href')
        response = requests.get(product_url)
        product_page = BeautifulSoup(response.content, 'html.parser')

        try:
            
            summary_div = product_page.find('div', class_ ="panel entry-content wc-tab")
            summary_txt = summary_div.find('p').get_text()
            img_url = product_page.find('div', class_ = "iconic-woothumbs-images-wrap").find('img').get('src')

            # Send a request to the image URL
            img_response = requests.get(img_url)

            # Get the image data
            # to do it in one line we can employ it as img_data = requests.get(product_page.find('div', class_ = "iconic-woothumbs-images-wrap").find('img').get('src')).content
            img_data = img_response.content

           
            # Split the text content into lines
            lines = summary_txt.split('\n')

            # Create a dictionary to store the extracted data
            data = {}
            for line in lines:
                if line:
                    key, value = line.split(':', 1)
                    data[key.strip()] = value.strip()

            csv_file_path = "/home/levi/Desktop/Work/Diamond Project/web_scraped/round/data_round.csv"

            # Write the data into a CSV file
            with open(csv_file_path, mode='a', newline='') as csv_file:
                fieldnames = data.keys()
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                # writer.writeheader()
                writer.writerow(data)

            print(f"Data saved to {csv_file_path}")

            # Save the image to a local file
            with open((f"web_scraped/round/images/{data['Stock Number']}.jpg") , "wb") as img_file:
                img_file.write(img_data)

            print("Image download and saving completed.")
        
        except:
            
            print("No image was found, hence no csv row was made, nor was a image saved.")


In [ ]:
base_url = "https://capitalwholesalediamonds.com/product-category/round/"
for i in range(1, 1411):
    if i == 1:
        base_url = "https://capitalwholesalediamonds.com/product-category/round/"
        page_iterator(base_url)
        print(f"Web scrapping page number:{i} - Completed")
    else:
        base_url = "https://capitalwholesalediamonds.com/product-category/round/"
        base_url = os.path.join(base_url, f'page/{i}/')
        page_iterator(base_url)
        print(f"Web scrapping page number:{i} - Completed")